# Simulation Design

#### This is the base file that:
- sets tiers
- sims each matchup
- simulates a single season



#### The functions in this file control
- the randomness of matchup win percentage
- changes in ELO after a win / loss
- which teams play eachother


In [2]:
import os
print(os.getcwd())

/Users/liameliot/Documents/DA 401/CFB


In [10]:
#Import packages
import numpy as np
from scipy.stats import norm
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import random

## Data Import

In [11]:
pnl = pd.read_csv('pnl.csv')
eloranks = pnl

## Set Tiers

In [5]:
def tiers(eloranks):
    tier1 = eloranks.head(34).reset_index(drop=True)
    eloranks = eloranks.tail(100)
    
    tier2 = eloranks.head(34).reset_index(drop=True)
    eloranks = eloranks.tail(66)
    
    tier3 = eloranks.head(34).reset_index(drop=True)
    eloranks = eloranks.tail(32)
    
    tier4 = eloranks.head(32).reset_index(drop=True)
    
    # Return tiers as a dictionary
    return {
        "tier1": tier1,
        "tier2": tier2,
        "tier3": tier3,
        "tier4": tier4
    }

# Calling the function and accessing tiers individually
tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]

## Resets season results to pnl format so that they can be reassigned by tiers function and antoher season can simulate

In [6]:
def sortelo(cfb24):
    #rank by elo
    pre25 = cfb24.sort_values(by='Week 12 ELO', ascending=False)
    #rename old columns
    pre25 = pre25.rename(columns={'Week 12 ELO': 'ELO'})
    #drop uneccesary columns
    pre25 = pre25.drop(columns=['Starting ELO', 'Week 12 Rank','Starting Rank','Wins','Losses','Tier'])
    #reset index
    pre25.reset_index(drop=True, inplace=True)
    
    preseason = pre25
    
    return preseason

## Sims Games

In [27]:
def simgame(team1, home_rating, team2, away_rating):
    # finds expected win percentage
    def get_expected_score(home_rating, away_rating):
        exp = (away_rating - home_rating + (globals().get("home_boost"))) / 400
        return 1 / (1 + 10**exp)

    # Calculate win probability for team1
    wp = get_expected_score(home_rating, away_rating) * 1000
    # Generate random number and simulate game outcome
    rn = random.uniform(0, 1000)
    rn1 = random.uniform(0, 5)
    rthing = globals().get("randomness")
    #margin = 1 if rn <= wp else 0
    margin = 1 if rn <= wp + rthing else 0
    
    # 1 if team1 wins, else 0

    # Determine the winner
    winner = team1 if margin == 1 else team2

    # Calculate new ELOs
    def get_new_elos(home_rating, away_rating, margin):
        
        k = globals().get("k_value")
        #k = 85
        home_score = margin
        away_score = 1 - home_score

        new_home_score = home_rating + k * (home_score - get_expected_score(home_rating, away_rating))
        new_away_score = away_rating + k * (away_score - get_expected_score(away_rating, home_rating))

        return round(new_home_score), round(new_away_score)

    new_home_elo, new_away_elo = get_new_elos(home_rating, away_rating, margin)

    return winner, new_home_elo, new_away_elo

## Sims games using altered k values

## Sims Season

In [28]:
def simulate_season(tier1):
    # Initialize the results DataFrame
    results = pd.DataFrame({
        'Team': tier1['Team'],
        'Wins': 0,
        'Losses': 0,
        'Starting ELO': tier1['ELO'],
        'Current ELO': tier1['ELO'],
        'Starting Rank': tier1['ELO'].rank(ascending=False, method='min').astype(int),
        'Home Games': 0,  # New column to count home games
        'Away Games': 0   # New column to count away games
    })

    # Prepare list to collect each week's matchups
    matchup_results = []

    # Simulate 12 weeks
    for week in range(12):
        # Rank teams based on their current Elo for matchups
        results['Current Rank'] = results['Current ELO'].rank(ascending=False, method='min').astype(int)
        results = results.sort_values(by='Current Rank').reset_index(drop=True)

        # Pair up teams based on ranking and simulate games
        for i in range(0, len(results), 2):
            if i + 1 < len(results):  # Ensure there's a team to pair with
                team1 = results.iloc[i]['Team']
                team2 = results.iloc[i + 1]['Team']
                
                # Determine home and away based on the number of home games each team has played
                if results.loc[i, 'Home Games'] < 6:
                    home_team, away_team = team1, team2
                    home_index, away_index = i, i + 1
                else:
                    home_team, away_team = team2, team1
                    home_index, away_index = i + 1, i

                home_rating = results.loc[home_index, 'Current ELO']
                away_rating = results.loc[away_index, 'Current ELO']

                # Simulate the game and determine the winner
                winner, new_home_elo, new_away_elo = simgame(home_team, home_rating, away_team, away_rating)

                # Update Wins and Losses
                if winner == home_team:
                    results.loc[home_index, 'Wins'] += 1
                    results.loc[away_index, 'Losses'] += 1
                else:
                    results.loc[home_index, 'Losses'] += 1
                    results.loc[away_index, 'Wins'] += 1

                # Update ELOs for both teams
                results.loc[home_index, 'Current ELO'] = new_home_elo
                results.loc[away_index, 'Current ELO'] = new_away_elo

                # Update home and away game counts
                results.loc[home_index, 'Home Games'] += 1
                results.loc[away_index, 'Away Games'] += 1

                # Record the matchup and result
                matchup_results.append({
                    'Week': week + 1,
                    'Matchup': f"{home_team} (Home) vs {away_team} (Away)",
                    'Winning Team': winner,
                    'Home Team ELO Before': home_rating,
                    'Away Team ELO Before': away_rating,
                    'CBR': abs(home_rating - away_rating)
                })

    # Add final ELO and ranking columns
    results['Week 12 ELO'] = results['Current ELO']
    results['Week 12 Rank'] = results['Current ELO'].rank(ascending=False, method='min').astype(int)

    # Final results DataFrame
    final_results = results[['Team', 'Wins', 'Losses', 'Starting ELO', 'Week 12 ELO', 'Starting Rank', 'Week 12 Rank', 'Home Games', 'Away Games']]

    # Matchup results DataFrame
    matchup_df = pd.DataFrame(matchup_results)

    return final_results, matchup_df


final_results, matchup_df = simulate_season(tier1)



TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'